# Import Libraries

In [1]:
import os
import sys
import pypots
import numpy as np
import benchpots
import matplotlib.pyplot as plt
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
from pypots.utils.random import set_random_seed
from functions.toolkits import toolkits
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from MAEModify.error import calc_mae
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012 as preprocess_physionet2012


2025-05-06 23:19:31.786802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746584371.802232   95830 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746584371.807081   95830 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 23:19:31.823459: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidge


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Load Dataset

In [2]:
set_random_seed()
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset.keys())

2025-05-06 23:19:45 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-05-06 23:19:45 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-05-06 23:19:45 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-05-06 23:19:45 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-05-06 23:19:45 [INFO]: Loaded successfully!
2025-05-06 23:19:58 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-06 23:19:58 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-05-06 23:19:58 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
dataset_for_training, dataset_for_validating, dataset_for_testing_ori, dataset_for_testing = toolkits.separating_dataset(physionet2012_dataset)

In [4]:
dataset_for_testing_ori= toolkits.dict_to_list(dataset_for_testing_ori)
dataset_for_testing = toolkits.dict_to_list(dataset_for_testing)

In [5]:
indicating_mask_subgroup, test_X_ori_subgroup = toolkits.components_mae(dataset_for_testing_ori, dataset_for_testing)

# Train/Load Models

## SAITS

In [6]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  
    MIT_weight=1,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-06 23:19:59 [INFO]: No given device, using default device: cpu
2025-05-06 23:19:59 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-06 23:19:59 [INFO]: Using customized MAE as the training loss function.
2025-05-06 23:19:59 [INFO]: Using customized MSE as the validation metric function.
2025-05-06 23:19:59 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


In [7]:
saits.load("../mae/tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots")

2025-05-06 23:19:59 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots


## BRITS

In [8]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-06 23:19:59 [INFO]: No given device, using default device: cpu
2025-05-06 23:19:59 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-06 23:19:59 [INFO]: Using customized MAE as the training loss function.
2025-05-06 23:19:59 [INFO]: Using customized MSE as the validation metric function.
2025-05-06 23:19:59 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


In [9]:
brits.load("../mae/tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots")

2025-05-06 23:19:59 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots


## US-GAN

In [10]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    epochs=10,
    patience=3,
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-06 23:19:59 [INFO]: No given device, using default device: cpu
2025-05-06 23:19:59 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-06 23:19:59 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


In [11]:
us_gan.load("../mae/tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots")

2025-05-06 23:19:59 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots


## GP-VAE

In [12]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-06 23:19:59 [INFO]: No given device, using default device: cpu
2025-05-06 23:19:59 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-06 23:19:59 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


In [13]:
gp_vae.load("../mae/tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots")

2025-05-06 23:19:59 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots


## MRNN

In [14]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    epochs=10,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    model_saving_strategy="best",
)

2025-05-06 23:19:59 [INFO]: No given device, using default device: cpu
2025-05-06 23:19:59 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-05-06 23:19:59 [INFO]: Using customized RMSE as the training loss function.
2025-05-06 23:19:59 [INFO]: Using customized MSE as the validation metric function.
2025-05-06 23:19:59 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


In [15]:
mrnn.load("../mae/tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots")

2025-05-06 23:19:59 [INFO]: Model loaded successfully from ../mae/tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots


# Imputation Models

## SAITS

In [16]:
saits_imputation = []
for i in range(len(dataset_for_testing)):
    _dict = {"X":dataset_for_testing[i]}
    saits_result = saits.predict(_dict)
    saits_imputation.append(saits_result['imputation'])

## BRITS

In [17]:
brits_imputation = []
for i in range(len(dataset_for_testing)):
    _dict = {"X":dataset_for_testing[i]}
    brits_result = brits.predict(_dict)
    brits_imputation.append(brits_result['imputation'])

## US-GAN

In [18]:
us_gan_imputation = []
for i in range(len(dataset_for_testing)):
    _dict = {"X":dataset_for_testing[i]}
    us_gan_result = us_gan.predict(_dict)
    us_gan_imputation.append(us_gan_result['imputation'])

## GP-VAE

In [19]:
gp_vae_imputation = []
for i in range(len(dataset_for_testing)):
    _dict = {"X":dataset_for_testing[i]}
    gp_vae_result = gp_vae.predict(_dict)
    gp_vae_imputation.append(gp_vae_result['imputation'])

## MRNN

In [20]:
mrnn_imputation = []
for i in range(len(dataset_for_testing)):
    _dict = {"X":dataset_for_testing[i]}
    mrnn_result = mrnn.predict(_dict)
    mrnn_imputation.append(mrnn_result['imputation'])

# AE/MAE Models

## SAITS

In [21]:
saits_mae = []
saits_ae = []

for i in range(len(saits_imputation)):
    aux_mae, aux_ae = calc_mae(saits_imputation[i], test_X_ori_subgroup[i], indicating_mask_subgroup[i])
    saits_mae.append(aux_mae)
    saits_ae.append(aux_ae)

## BRITS

In [22]:
brits_mae = []
brits_ae = []

for i in range(len(brits_imputation)):
    aux_mae, aux_ae = calc_mae(brits_imputation[i], test_X_ori_subgroup[i], indicating_mask_subgroup[i])
    brits_mae.append(aux_mae)
    brits_ae.append(aux_ae)

## USGAN

In [23]:
us_gan_mae = []
us_gan_ae = []

for i in range(len(us_gan_imputation)):
    aux_mae, aux_ae = calc_mae(us_gan_imputation[i], test_X_ori_subgroup[i], indicating_mask_subgroup[i])
    us_gan_mae.append(aux_mae)
    us_gan_ae.append(aux_ae)

## GPVAE

In [24]:
gp_vae_mae = []
gp_vae_ae = []

for i in range(len(gp_vae_imputation)):
    gp_vae_imputation_fix = np.squeeze(gp_vae_imputation[i], axis=1)
    aux_mae, aux_ae = calc_mae(gp_vae_imputation_fix, test_X_ori_subgroup[i], indicating_mask_subgroup[i])
    gp_vae_mae.append(aux_mae)
    gp_vae_ae.append(aux_ae)

## MRNN

In [25]:
mrnn_mae = []
mrnn_ae = []

for i in range(len(mrnn_imputation)):
    aux_mae, aux_ae = calc_mae(mrnn_imputation[i], test_X_ori_subgroup[i], indicating_mask_subgroup[i])
    mrnn_mae.append(aux_mae)
    mrnn_ae.append(aux_ae)

# Gini Models

## SAITS

In [26]:
saits_ginis = []

for i in range(len(saits_ae)):
    saits_ae[i] = saits_ae[i].reshape(len(saits_ae[i])*48*37)
    saits_ginis.append(toolkits.gini(saits_ae[i]))

In [27]:
saits_ginis

[np.float64(0.5487214671675433),
 np.float64(0.5472045375576453),
 np.float64(0.5503036848476004),
 np.float64(0.5176627079692269),
 np.float64(0.5474375794148922),
 np.float64(0.5502383614631714),
 np.float64(0.5435888878847976),
 np.float64(0.5489472716984587),
 np.float64(0.549011875368939),
 np.float64(0.5508369755105551),
 np.float64(0.545940039442044),
 np.float64(0.5513664140608352),
 np.float64(0.5479675203931538),
 np.float64(0.5515527600631114),
 np.float64(0.5548843022508989),
 np.float64(0.5509181832728041),
 np.float64(0.5608951955407246)]

## BRITS

In [28]:
brits_ginis = []

for i in range(len(brits_ae)):
    brits_ae[i] = brits_ae[i].reshape(len(brits_ae[i])*48*37)
    brits_ginis.append(toolkits.gini(brits_ae[i]))

In [29]:
brits_ginis

[np.float64(0.592741084674985),
 np.float64(0.5909028925193482),
 np.float64(0.5941360031276413),
 np.float64(0.573110285137687),
 np.float64(0.5935625131232738),
 np.float64(0.5915644696916396),
 np.float64(0.586939353263334),
 np.float64(0.5980453048184342),
 np.float64(0.5965319845743837),
 np.float64(0.5863151906863731),
 np.float64(0.5891037341666552),
 np.float64(0.5862304793940845),
 np.float64(0.5955328715473043),
 np.float64(0.595556255533837),
 np.float64(0.5956846752251219),
 np.float64(0.5941990518260556),
 np.float64(0.6043933960684573)]

## US-GAN

In [30]:
us_gan_ginis = []

for i in range(len(us_gan_ae)):
    us_gan_ae[i] = us_gan_ae[i].reshape(len(us_gan_ae[i])*48*37)
    us_gan_ginis.append(toolkits.gini(us_gan_ae[i]))

In [31]:
us_gan_ginis

[np.float64(0.6530495247432959),
 np.float64(0.6499284175702373),
 np.float64(0.6554331274054435),
 np.float64(0.6038760308991644),
 np.float64(0.6514461138488092),
 np.float64(0.6550339384040282),
 np.float64(0.6390442305433826),
 np.float64(0.6723713903217969),
 np.float64(0.6438886161059825),
 np.float64(0.6547659878586771),
 np.float64(0.6435542611787677),
 np.float64(0.6538218400212623),
 np.float64(0.6601800617722127),
 np.float64(0.6629244512427519),
 np.float64(0.6632810162294351),
 np.float64(0.6595838071145177),
 np.float64(0.6608289912281815)]

## GP-VAE

In [32]:
gp_vae_ginis = []

for i in range(len(gp_vae_ae)):
    gp_vae_ae[i] = gp_vae_ae[i].reshape(len(gp_vae_ae[i])*48*37)
    gp_vae_ginis.append(toolkits.gini(gp_vae_ae[i]))

In [33]:
gp_vae_ginis

[np.float64(0.6139831641630205),
 np.float64(0.6125614039524674),
 np.float64(0.6156229027898539),
 np.float64(0.5975784227555638),
 np.float64(0.6110162952036104),
 np.float64(0.6169350117926335),
 np.float64(0.6022356398221163),
 np.float64(0.61319289299411),
 np.float64(0.6145132135818177),
 np.float64(0.6186603797818135),
 np.float64(0.6129241816060156),
 np.float64(0.6192004468919334),
 np.float64(0.6154260693522435),
 np.float64(0.6178367139036256),
 np.float64(0.6142979929362999),
 np.float64(0.6121548482848884),
 np.float64(0.6128499487405066)]

## MRNN

In [34]:
mrnn_ginis = []

for i in range(len(mrnn_ae)):
    mrnn_ae[i] = mrnn_ae[i].reshape(len(mrnn_ae[i])*48*37)
    mrnn_ginis.append(toolkits.gini(mrnn_ae[i]))

In [35]:
mrnn_ginis

[np.float64(0.33234427376132825),
 np.float64(0.3346662178739479),
 np.float64(0.33066728151191027),
 np.float64(0.23307058739225073),
 np.float64(0.33220366056644623),
 np.float64(0.3332104711467936),
 np.float64(0.31249969348744444),
 np.float64(0.33216280632564077),
 np.float64(0.33401597306046726),
 np.float64(0.341832868892453),
 np.float64(0.32519029309308367),
 np.float64(0.35521619795764225),
 np.float64(0.337419930291202),
 np.float64(0.33823532510101284),
 np.float64(0.33604543563135647),
 np.float64(0.33395311518926496),
 np.float64(0.35137757823445925)]